In [1]:
import pandas as pd

data = pd.read_csv('Data/raw_data.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'Data/raw_data.csv'

In [3]:
data.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [6]:
resume_text = data.iloc[0].Resume

In [8]:
resume_text

'Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\n\r\nData Science Assurance Associate \r\n\r\nData Science Assurance Associate - Ernst & Young LLP\r\nSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigatio

In [ ]:
import re
from collections import Counter

# Regex patterns
TITLE_PATTERN = re.compile(r"\b(?:Data Scientist|Data Science Assurance Associate|Machine Learning Engineer|Software Engineer|Developer|Engineer|Analyst|Consultant)\b", re.IGNORECASE)
SKILLS_PATTERN = re.compile(r"(Skills.*?:|Tools & Technologies:|Skill Details)(.*?)(?=(?:Education Details|Company Details|$))", re.IGNORECASE | re.DOTALL)
EDU_PATTERN = re.compile(r"(?:B\.?E\.?|B\.?Tech|M\.?Tech|Bachelor|Master|Ph\.?D|Diploma).*?(?=\s{2,}|\n|$)", re.IGNORECASE)
YOE_PATTERN = re.compile(r"(\d+)\s*\+?\s*(?:years|yrs|yoe|experience)", re.IGNORECASE)
EXPERIENCE_LINE_PATTERN = re.compile(r"(?:Experience\s*-\s*)(\d+)\s*(?:months|years)", re.IGNORECASE)

def extract_title(text):
    titles = TITLE_PATTERN.findall(text)
    return titles[0] if titles else ""

def extract_skills(text):
    skills_found = []
    for match in SKILLS_PATTERN.findall(text):
        skills_text = match[1]
        # Split on commas, semicolons, and newlines
        parts = re.split(r"[,\n;]", skills_text)
        skills_found.extend([p.strip() for p in parts if p.strip()])
    # Remove duplicates while preserving order
    seen = set()
    skills_clean = []
    for skill in skills_found:
        if skill.lower() not in seen:
            seen.add(skill.lower())
            skills_clean.append(skill)
    return skills_clean

def extract_education(text):
    return list(set(EDU_PATTERN.findall(text)))

def infer_seniority(text):
    # Gather years from patterns
    years = [int(y) for y in YOE_PATTERN.findall(text)]
    months = [int(m) for m in EXPERIENCE_LINE_PATTERN.findall(text)]
    total_years = sum(years) + sum(months) / 12
    # Infer seniority
    if total_years >= 7:
        level = "Senior"
    elif total_years >= 3:
        level = "Mid-level"
    elif total_years > 0:
        level = "Junior"
    else:
        level = ""
    return level, round(total_years, 1)

def extract_cv_fields(cv_text):
    return {
        "title": extract_title(cv_text),
        "skills_tech_stack": extract_skills(cv_text),
        "educational_requirements": extract_education(cv_text),
        "seniority": infer_seniority(cv_text)[0],
        "total_years_experience": infer_seniority(cv_text)[1]
    }


{'title': 'Data Science Assurance Associate', 'skills_tech_stack': ['Python (pandas', 'numpy', 'scipy', 'scikit-learn', 'matplotlib)', 'Sql', 'Java', 'JavaScript/JQuery. * Machine learning: Regression', 'SVM', 'NaÃ¯ve Bayes', 'KNN', 'Random Forest', 'Decision Trees', 'Boosting techniques', 'Cluster Analysis', 'Word Embedding', 'Sentiment Analysis', 'Natural Language processing', 'Dimensionality reduction', 'Topic Modelling (LDA', 'NMF)', 'PCA & Neural Nets. * Database Visualizations: Mysql', 'SqlServer', 'Cassandra', 'Hbase', 'ElasticSearch D3.js', 'DC.js', 'Plotly', 'kibana', 'matplotlib', 'ggplot', 'Tableau. * Others: Regular Expression', 'HTML', 'CSS', 'Angular 6', 'Logstash', 'Kafka', 'Python Flask', 'Git', 'Docker', 'computer vision - Open CV and understanding of Deep learning.', 'JAVASCRIPT- Exprience - 24 months', 'jQuery- Exprience - 24 months', 'Python- Exprience - 24 months', 'Python', 'tfidf', 'word2vec', 'doc2vec', 'cosine similarity', 'LDA', 'NMF for topic modelling', 'Vad

In [ ]:
import pandas as pd
import uuid
import random

# Example name pool for random generation
FIRST_NAMES = ["Alex", "Jordan", "Taylor", "Casey", "Morgan", "Jamie", "Riley", "Avery", "Reese", "Drew"]
LAST_NAMES = ["Smith", "Johnson", "Taylor", "Brown", "Anderson", "Lee", "Martinez", "Davis", "Clark", "Walker"]

def random_name():
    return f"{random.choice(FIRST_NAMES)} {random.choice(LAST_NAMES)}"

def random_id():
    return str(uuid.uuid4())[:8]  # short unique ID

# Store results
results = []

for i in range(len(data)):
    resume_text = str(data.iloc[i].Resume)  # ensure string
    extracted = extract_cv_fields(resume_text)
    results.append({
        "candidate_id": random_id(),
        "name": random_name(),
        "candidate_title": extracted["title"],
        "education": ", ".join(extracted["educational_requirements"]),
        "skills_tech_stack": ", ".join(extracted["skills_tech_stack"]),
        "seniority": extracted["seniority"],
        "total_years_experience": extracted["total_years_experience"]
    })

# Save to CSV
df_results = pd.DataFrame(results)
df_results.to_csv("cv_dataset.csv", index=False, encoding="utf-8")

print("✅ Extracted data saved to cv_dataset.csv")


✅ Extracted data saved to cv_dataset.csv


In [3]:
import re
import random
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()

def generate_random_email(name):
    return f"{name.lower().replace(' ', '.')}@{fake.free_email_domain()}"

def calculate_yoe(text):
    matches = re.findall(r'(\d+)\s*(?:\+?\s*)?(?:years?|yrs?)', text, flags=re.IGNORECASE)
    years = [int(m) for m in matches]
    return max(years) if years else 0

def infer_seniority(yoe):
    if yoe < 2:
        return "Junior"
    elif yoe < 5:
        return "Mid-level"
    elif yoe < 10:
        return "Senior"
    else:
        return "Lead/Principal"

def extract_candidate_title(text):
    # Look for patterns like "Data Scientist", "Software Engineer", etc.
    match = re.search(r'(?:Title|Position|Role|Designation)\s*[:-]?\s*(.+)', text, re.IGNORECASE)
    if match:
        return match.group(1).split('\n')[0].strip()
    # Try job title-like pattern from first few lines
    lines = text.strip().split("\n")
    for line in lines[:5]:
        if any(word.lower() in line.lower() for word in ["engineer", "scientist", "developer", "manager", "analyst"]):
            return line.strip()
    return ""

def extract_about(text):
    # Take the first ~500 chars as a summary
    about = re.sub(r'\s+', ' ', text.strip())
    return about[:500] + ("..." if len(about) > 500 else "")

def extract_employment_type(text):
    types = ["full-time", "part-time", "contract", "internship", "freelance", "temporary"]
    for t in types:
        if re.search(t, text, re.IGNORECASE):
            return t.capitalize()
    return ""

def extract_cv_fields(cv_text):
    full_name = fake.name()
    email = generate_random_email(full_name)
    location = f"{fake.city()}, {fake.country()}"
    created_at = fake.date_time_between(start_date="-2y", end_date="now").isoformat()

    yoe = calculate_yoe(cv_text)
    seniority = infer_seniority(yoe)
    title = extract_candidate_title(cv_text)
    about = extract_about(cv_text)
    emp_type = extract_employment_type(cv_text)

    return {
        "candidate_email": email,
        "full_name": full_name,
        "candidate_title": title,
        "about": about,
        "location": location,
        "preferred_employment_type": emp_type,
        "yoe": yoe,
        "seniority": seniority,
        "created_at": created_at
    }

# Example: process all CVs in a DataFrame
def process_resumes_to_csv(df, output_file="candidates_parsed.csv"):
    results = []
    for i in range(len(df)):
        parsed = extract_cv_fields(df.iloc[i].Resume)
        results.append(parsed)
    pd.DataFrame(results).to_csv(output_file, index=False, encoding="utf-8")
    print(f"Saved {len(results)} candidates to {output_file}")

# Example usage:
# data = pd.read_csv("resumes.csv")  # must have 'Resume' column
# process_resumes_to_csv(data)


In [2]:
!pip install faker --quiet


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
data = pd.read_csv("../Data/raw_data_cvs.csv")  # must have 'Resume' column
process_resumes_to_csv(data)

Saved 962 candidates to candidates_parsed.csv
